# Text Mining Group Project
## ZH-EN Corpus

##### TOC for Implemented Metrics.

---

In [1]:
# Necessary Installs.
#!pip install rouge

In [2]:
# Imports
import pandas as pd
import numpy as np
from collections import Counter
from rouge import Rouge
import string
from nltk.translate import chrf_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import StandardScaler

Load Dataset

In [3]:
df1 = pd.read_csv("scores.csv")

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26419 entries, 0 to 26418
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   source       26419 non-null  object 
 1   reference    26419 non-null  object 
 2   translation  26419 non-null  object 
 3   z-score      26419 non-null  float64
 4   avg-score    26419 non-null  float64
 5   annotators   26419 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 1.2+ MB


In [5]:
df1.head()

,source,reference,translation,z-score,avg-score,annotators
0,他性格活泼，这对英国赛马来说是好事，但是除此之外，他还是一位不可思议的骑师。,"His character is good for the British horse, b...",He's a lively character which is good for Brit...,0.625559,92.75,4
1,近日刚搬至旧金山的一位28岁厨师本周被发现死于当地一家商场的楼梯间。,"A 28 chef, who has just moved to San Francisco...",A 28-year-old chef who had recently moved to S...,0.550952,92.00,4
2,去年，有官员表示，胡克先生的团队所得出的结论是针对伊斯兰国炼油厂的空袭并未大幅削减恐怖组织的...,"Last year, officials said Mr. Hooker's team ha...","Last year, officials said, Mr. Hooker's team c...",0.540814,89.00,5
3,尤其值得玩味的是政府对于饥饿民众们的回应，比如总统市民赫伯特·胡佛“别人的事我可管不了”的态度。,It is particularly interesting to note the gov...,Especially savory are the accounts of the gove...,-0.793944,49.50,4
4,不过，从20世纪90年代至今，人类共进行了18次火星探测，而月球探测只进行了14次。,"However, from the 1990s to the present, human ...","However, ever since the 1990s, a total of 18 h...",0.046532,77.50,4


---
### PreProcessing

In [6]:
# Check for empty or sparse reference / translation, and drop them.
for column in ["source","reference","translation"]:
    print(column)
    bad_idx = [idx for idx in np.where(df1[column].str.len()<=1)[0]]
    if bad_idx != []:
        print(df1.iloc[bad_idx])
    print(f"Bad idx: {bad_idx}")
    df1 = df1.drop(index=bad_idx)

source
Bad idx: []
reference
                                                  source reference  \
15223  根据《政客新闻网》的报道，特朗普在上世纪九十年代有两年几乎没有缴纳过一分钱的税，《纽约时报》...         .   

                                             translation   z-score  avg-score  \
15223  Politico has reported that Trump paid zero to ...  0.696136      100.0   

       annotators  
15223           1  
Bad idx: [15223]
translation
Bad idx: []


In [7]:
# Create two dataframes. One is unprocessed, the other is preprocessed to remove punctuation and be lowercased.
# (Future note: maybe also remove stopwords?)
df_u = df1.copy()
df_p = df1.copy()

for x in ["source","reference","translation"]:
    # lowercase.
    df_p[x] = df1[x].str.lower()
    # Remove punct.
    df_p[x] = df1[x].map(lambda s: s.translate(str.maketrans("","",string.punctuation))
                           .lower()
                          )
    
df_dict = {"df_u":df_u,"df_p":df_p}

In [8]:
## Initialize a scaler for later.
scaler = StandardScaler()

---
### Metrics

--- 
Bleu Metric

In [9]:
def my_first_BLEU(reference,translation):
    """
    Expects lists of strings for both reference and translation.
    Returns the score 
    """
    
    # Let word be every unique word in the translation.
    # Can be done by setting up a Counter object.
    t_c = Counter(word_tokenize(translation))
    words = sorted(t_c)
    
    refs_c = Counter(word_tokenize(reference))
    
    # Let Covered be the minimum amt of times a word appears in the reference, compared to R(w).
    # Let D(word) be how many times a unique word appears in the candidate translation.
    # Let R(word) be the largest numer of times the word appears in any one reference.
    
    covered = 0
    
    for word in words:
        covered += min(t_c[word],refs_c[word])

    

    # Let total be the number of words in translation.
    total = sum(t_c.values())
    
    BLEU_score = covered / total
    
    return BLEU_score


In [10]:
bleu_scores_list = ["basic bleu"]

for df in list(df_dict.values()):
    for key in bleu_scores_list:
        # Apply the function to get a column of the scores.
        df[(key+" score")] = df.apply(lambda row: my_first_BLEU(row["reference"],row["translation"]),axis=1)
        # Also add a z score column.
        df[(key+ "_zscore")] = scaler.fit_transform(df[(key+" score")].to_numpy().reshape(-1,1)).flatten()

---
Rouge metric as described in

https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460

And
https://pypi.org/project/rouge-metric/

In [11]:
rouge = Rouge()

In [12]:
# Example Cell
model_out = df1["translation"][0]
reference = df1["reference"][0]
# The get scores method returns three metrics, F1 score, p precision and recall r.
# For each unigram,bigram and Longest sequence.
rouge.get_scores(model_out,reference)

[{'rouge-1': {'f': 0.5294117597231834, 'p': 0.5, 'r': 0.5625},
  'rouge-2': {'f': 0.24999999501953132,
   'p': 0.23529411764705882,
   'r': 0.26666666666666666},
  'rouge-l': {'f': 0.5454545404958678, 'p': 0.5, 'r': 0.6}}]

In [13]:
# Get scores.
# For every df considered;
for df in list(df_dict.values()):
    # For the entire model, model_out and reference need to be lists of strings.
    model_out = df["translation"].to_list()
    reference = df["reference"].to_list()
    rouge_scores = rouge.get_scores(model_out,reference)
    # For each of the scores calculated, output a new column in the df with the f1 scores.
    for key in rouge_scores[0].keys():
        df[(key+" score")] = pd.Series([score[key]["f"] for score in rouge_scores])
        # Also add a z score column.
        df[(key+ "_zscore")] = scaler.fit_transform(df[(key+" score")].to_numpy().reshape(-1,1)).flatten()

---
#### chrF metric

Check the paper here: https://www.aclweb.org/anthology/W15-3049.pdf

The general formula for the CHRF score is:

`CHRFBeta = (1 + Beta**2) * ((chrP * chrR) / (Beta**2*chrP + chrR))`

where:
* chrP is the percentage of n-grams in the hypothesis which have a counterpart in the reference.
* chrR is the percentage of character n-grams in the reference which are also present in the hypothesis.
* Beta is a parameter which assigns beta times more importance to recall than to precision (if beta == 1, they have the same importance).

In [14]:
# I was surprised, but this works exactly like it's intended. Makes a new column with the chrF score for each row of the df.
# The default n-gram values are min == 1, max == 6. 
# The default beta is 3.

# Moreover, it is worthwhile to mention chrf uses its own tokenization with whitespaces.
# All parameters to test chrf scores with. feel free to play around with this and test out different combinations.
# Note: this takes a few minutes to run.
min_len = [1,2]
max_len = [6,10]
beta = [1,3]

for df in list(df_dict.values()):
    chrf_scores=[]
    for min_l in min_len:
        for max_l in max_len:
            for b in beta:
                append_str = "chrf_b" + str(b) + "_n" + str(min_l) + str(max_l)
                chrf_scores.append(append_str)
                df[append_str] = df.apply(lambda row: chrf_score.sentence_chrf(row["reference"],row["translation"],min_len=min_l,max_len=max_l,beta=b),axis=1)
                # Also add a z score column.
                df[(append_str+ "_zscore")] = scaler.fit_transform(df[append_str].to_numpy().reshape(-1,1)).flatten()

df_p.loc[:,chrf_scores]

,chrf_b1_n16,chrf_b3_n16,chrf_b1_n110,chrf_b3_n110,chrf_b1_n26,chrf_b3_n26,chrf_b1_n210,chrf_b3_n210
0,0.527695,0.560819,0.401800,0.427455,0.463299,0.492677,0.352037,0.374780
1,0.720804,0.772510,0.630924,0.677172,0.692871,0.742933,0.605419,0.650147
2,0.589640,0.550085,0.476770,0.444627,0.532237,0.496447,0.432338,0.403110
3,0.477323,0.474164,0.394625,0.391979,0.394099,0.391472,0.339200,0.336907
4,0.400072,0.401561,0.267354,0.268354,0.313877,0.315054,0.204721,0.205494
...,...,...,...,...,...,...,...,...
26414,0.666128,0.672838,0.536816,0.542286,0.619761,0.626037,0.496689,0.501780
26415,0.449596,0.488016,0.330611,0.359332,0.377871,0.410545,0.277543,0.301994
26416,0.311032,0.334206,0.199038,0.214064,0.248648,0.267714,0.151936,0.163774
26417,0.181584,0.166995,0.108950,0.100197,0.091585,0.083889,0.050881,0.046605


---
### Comparison of Applied Metrics
Because the numeric system used for all of these can be different, the best way to compare them is by checking the correlation with the annotator's scores.

In [15]:
# Initialize a dict to be transformed to a df later, for score comparison.
corr_list = ["pearson","kendall"]
scores_dict = {}

for df_name in df_dict.keys():
    for corr in corr_list:
        scores_dict[corr+"_"+df_name] = []
    
scores_index = []

In [16]:
# Thankfully, Pandas has a corr method. Use it on standardized scores obtained previously.

# for each declared corr method, compute the corr between each computed metric and the avg-score column for each considered df.
for df_name in df_dict.keys():
    for corr in corr_list:
        for key in rouge_scores[0].keys():
            scores_dict[corr+"_"+df_name].append(df_dict[df_name].loc[:,(key+ "_zscore")].corr(df_dict[df_name].loc[:,"z-score"],method=corr))
        for chrf_score in set(chrf_scores):
            scores_dict[corr+"_"+df_name].append(df_dict[df_name].loc[:,(chrf_score+"_zscore")].corr(df_dict[df_name].loc[:,"z-score"],method=corr))
        for bleu_score in bleu_scores_list:
            scores_dict[corr+"_"+df_name].append(df_dict[df_name].loc[:,(key+ "_zscore")].corr(df_dict[df_name].loc[:,"z-score"],method=corr))


# Build also a list that will be used to create the index for the scores dataframe.
scores_index.extend(list(rouge_scores[0].keys()))
scores_index.extend(chrf_scores)
scores_index.extend(bleu_scores_list)

In [17]:
scores_df = pd.DataFrame(scores_dict,index=scores_index)
scores_df

,pearson_df_u,kendall_df_u,pearson_df_p,kendall_df_p
rouge-1,0.205612,0.138001,0.202819,0.135179
rouge-2,0.196401,0.130561,0.193723,0.128462
rouge-l,0.208347,0.140101,0.209769,0.139644
chrf_b1_n16,0.324004,0.214737,0.318096,0.209368
chrf_b3_n16,0.299939,0.197765,0.292487,0.191504
chrf_b1_n110,0.302165,0.197806,0.292559,0.190488
chrf_b3_n110,0.333166,0.220173,0.326618,0.214780
chrf_b1_n26,0.320758,0.213059,0.315422,0.208019
chrf_b3_n26,0.301502,0.199535,0.294745,0.193677
chrf_b1_n210,0.336367,0.222059,0.329069,0.216233


In [18]:
for column in scores_df.columns:
    print(f"Max in {column}:")
    print("{}, with {}".format(scores_df[column].argmax(),scores_df[column].max()))


Max in pearson_df_u:
9, with 0.33636712080717346
Max in kendall_df_u:
9, with 0.22205935427807688
Max in pearson_df_p:
9, with 0.3290691571404494
Max in kendall_df_p:
9, with 0.21623336539883367
